In [4]:
import os, sys
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path)

import pandas as pd
import numpy as np

# !pip install emoji==1.6.3
# !pip install scipy
from pygents.util import calc_f1

## Cursory check of LLM capacity to detect distortions

In [5]:
from langchain_ollama.chat_models import ChatOllama
default_chat_model = "llama3.2"

llm = ChatOllama(model=default_chat_model, base_url="http://localhost:11434")  # Explicitly set base_url

In [18]:
def evaluator_llm(text,threshold=0):
    query = f"Be concise. Does this text have cognitive distortions in it \"{text}\"?"
    r = llm.invoke(query).content
    return r.lower().startswith("yes")


In [19]:
texts = [
    "I am such a failure I never do anything right.",
    "I am a software developer doing coding.",
    "I am a man sitting on the chair behind the table.",
    "There is a chair behind the table."
]
for r in texts:
    q = f"Be concise. Does this text have cognitive distortions in it \"{r}\"?"
    d = evaluator_llm(r)
    print(f"{d}: {q}")
    
    

True: Be concise. Does this text have cognitive distortions in it "I am such a failure I never do anything right."?
True: Be concise. Does this text have cognitive distortions in it "I am a software developer doing coding."?
True: Be concise. Does this text have cognitive distortions in it "I am a man sitting on the chair behind the table."?
False: Be concise. Does this text have cognitive distortions in it "There is a chair behind the table."?


## Explore performance of "our out of the box" model with dataset 1 (original binary)

In [20]:
binary_dataset_file_path = "./data/corpora/English/distortions/halilbabacan/raw_Cognitive_distortions.csv" 
df = pd.read_csv(binary_dataset_file_path)
df.insert(1, "N/A text", value = np.nan)
df.insert(3, "N/A label", value = np.nan)
df.head(10)


,Text,N/A text,Label,N/A label
0,I'm such a failure I never do anything right.,NaN,Distortion,NaN
1,Nobody likes me because I'm not interesting.,NaN,Distortion,NaN
2,I can't try new things because I'll just mess...,NaN,Distortion,NaN
3,My boss didn't say 'good morning' she must be...,NaN,Distortion,NaN
4,My friend didn't invite me to the party I mus...,NaN,Distortion,NaN
5,I didn't get the job so I must be incompetent.,NaN,Distortion,NaN
6,I'm always unlucky. Good things only happen t...,NaN,Distortion,NaN
7,Everyone thinks I'm stupid because I made a m...,NaN,Distortion,NaN
8,I'll never be successful because I failed my ...,NaN,Distortion,NaN
9,Nobody cares about me because they didn't ask...,NaN,Distortion,NaN


In [21]:
from pygents.aigents_api import TextMetrics
#from pygents.util import vector_proximity


def language_metrics(lang,metrics_list):
    metrics = {}
    for m in metrics_list:
        metrics[m] = './data/dict/' + lang + '/' + m + '.txt'
    return metrics


distortion_labels = ['positive','negative','rude',
'catastrophizing','dichotomous-reasoning','disqualifying-positive','emotional-reasoning','fortune-telling',
'labeling','magnification','mental-filtering','mindreading','overgeneralizing','personalizing','should-statement']
tm = TextMetrics(language_metrics('en',distortion_labels),debug=False)

def f1_from_counts(true_positive, true_negative, false_positive, false_negative):
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    return 2 * precision * recall / (precision + recall) if precision > 0 or recall > 0 else 0 

def evaluate_df(df,evaluator,threshold,debug=False):
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    for _, row in df.iterrows():
        # Text definition: first, check the 2nd column; if NaN, take the text from the 1st column.
        text = row.iloc[1] if pd.notna(row.iloc[1]) else row.iloc[0]
        primary_distortion = row.iloc[2]  # The main cognitive distortion from the 3rd column
        secondary_distortion = row.iloc[3] if pd.notna(row.iloc[3]) else None  # The secondary distortion from the 4th column, if it exists
        ground_distortion = False if primary_distortion == 'No Distortion' else True
                       
        our_distortion = evaluator(text,threshold)
        
        # https://en.wikipedia.org/wiki/F-score
        if ground_distortion == True and our_distortion == True:
            true_positive += 1
        if ground_distortion == False and our_distortion == True:
            false_positive += 1
        if ground_distortion == False and our_distortion == False:
            true_negative += 1
        if ground_distortion == True and our_distortion == False:
            false_negative += 1

        if debug:
            print(ground_distortion,our_distortion,text[:20],metrics)

    return f1_from_counts(true_positive, true_negative, false_positive, false_negative) 


def our_evaluator_any(text,threshold):
    metrics = tm.get_sentiment_words(text)
    for m in metrics:
        if metrics[m] > threshold:
            return True
    return False

def our_evaluator_avg(text,threshold):
    metrics = tm.get_sentiment_words(text)
    l = list(metrics.values())
    avg = sum(l) / len(l) if  len(l) > 0 else 0
    if avg > threshold:
        return True
    return False
  

In [22]:
for threshold in [0.0,0.01,0.05,0.1,0.2,0.4,0.6,0.8]:
    f1 = evaluate_df(df,our_evaluator_any,threshold)
    print(threshold, f1)

0.0 0.8443643512450851
0.01 0.8443643512450851
0.05 0.8443643512450851
0.1 0.8443643512450851
0.2 0.8445027035883992
0.4 0.8444669365721997
0.6 0.5569898379566054
0.8 0.013021830716200687


In [23]:
for threshold in [0.0,0.01,0.05,0.1,0.2,0.4,0.6,0.8]:
    f1 = evaluate_df(df,our_evaluator_avg,threshold)
    print(threshold, f1)

0.0 0.8443643512450851
0.01 0.8443643512450851
0.05 0.8443643512450851
0.1 0.8443643512450851
0.2 0.8447795443369939
0.4 0.886836935166994
0.6 0.18404478656403078
0.8 0.007680491551459293
